In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import string

In [2]:
import re
import nltk

In [3]:
train = pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [4]:
train['text_len'] = train.text.map(lambda x : len(x))
test['text_len'] = test.text.map(lambda x : len(x))

In [5]:
train

,id,keyword,location,text,target,text_len
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,83
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,125
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,137


In [6]:
train=train.drop('id',axis=1)
test=test.drop('id',axis=1)

In [7]:
train=train.drop('location',axis=1)
test=test.drop('location',axis=1)

In [8]:
from nltk.corpus import stopwords

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus_train = []
for i in range(0, len(train)):
  review = re.sub('[^a-zA-Z]', ' ', train['text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus_train.append(review)

[nltk_data] Downloading package stopwords to C:\Users\ZAIDI Mohamed
[nltk_data]     Arysse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus_test = []
for i in range(0, len(test)):
  review = re.sub('[^a-zA-Z]', ' ', test['text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus_test.append(review)

[nltk_data] Downloading package stopwords to C:\Users\ZAIDI Mohamed
[nltk_data]     Arysse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
train['text'] = corpus_train
test['text'] = corpus_test

In [12]:
train

,keyword,text,target,text_len
0,NaN,deed reason earthquak may allah forgiv us,1,69
1,NaN,forest fire near la rong sask canada,1,38
2,NaN,resid ask shelter place notifi offic evacu she...,1,133
3,NaN,peopl receiv wildfir evacu order california,1,65
4,NaN,got sent photo rubi alaska smoke wildfir pour ...,1,88
...,...,...,...,...
7608,NaN,two giant crane hold bridg collaps nearbi home...,1,83
7609,NaN,aria ahrari thetawniest control wild fire cali...,1,125
7610,NaN,utc km volcano hawaii http co zdtoyd ebj,1,65
7611,NaN,polic investig e bike collid car littl portug ...,1,137


In [13]:
train ['keyword'] = train['keyword'].fillna(value='fatalities')
test ['keyword'] = test['keyword'].fillna(value='fatalities')

In [14]:
def remove_URL (text) :
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r"",text)

def remove_HTML(text) :
    html=re.compile(r"<.*?>")
    return html.sub(r"", text)

In [15]:
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

In [16]:
def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [17]:
train["text"] = train.text.map(lambda x: remove_URL(x))
train["text"] = train.text.map(lambda x: remove_HTML(x))
train["text"] = train.text.map(lambda x: remove_emoji(x))
train["text"] = train.text.map(lambda x: remove_punct(x))

In [18]:
test["text"] = test.text.map(lambda x: remove_URL(x))
test["text"] = test.text.map(lambda x: remove_HTML(x))
test["text"] = test.text.map(lambda x: remove_emoji(x))
test["text"] = test.text.map(lambda x: remove_punct(x))

In [19]:
train

,keyword,text,target,text_len
0,fatalities,deed reason earthquak may allah forgiv us,1,69
1,fatalities,forest fire near la rong sask canada,1,38
2,fatalities,resid ask shelter place notifi offic evacu she...,1,133
3,fatalities,peopl receiv wildfir evacu order california,1,65
4,fatalities,got sent photo rubi alaska smoke wildfir pour ...,1,88
...,...,...,...,...
7608,fatalities,two giant crane hold bridg collaps nearbi home...,1,83
7609,fatalities,aria ahrari thetawniest control wild fire cali...,1,125
7610,fatalities,utc km volcano hawaii http co zdtoyd ebj,1,65
7611,fatalities,polic investig e bike collid car littl portug ...,1,137


In [20]:
test

,keyword,text,text_len
0,fatalities,happen terribl car crash,34
1,fatalities,heard earthquak differ citi stay safe everyon,64
2,fatalities,forest fire spot pond gees flee across street ...,96
3,fatalities,apocalyps light spokan wildfir,40
4,fatalities,typhoon soudelor kill china taiwan,45
...,...,...,...
3258,fatalities,earthquak safeti lo angel safeti fasten xrwn,55
3259,fatalities,storm ri wors last hurrican citi amp other har...,139
3260,fatalities,green line derail chicago http co utbxlcbiuy,55
3261,fatalities,meg issu hazard weather outlook hwo http co x ...,65


In [21]:
from collections import Counter

def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [22]:
text = train.text

counter = counter_word(text)

In [23]:
len(counter)

18890

In [24]:
num_words = len(counter)
max_length = 20

In [25]:
train_size = int(train.shape[0] * 0.8)

train_sentences = train.text[:train_size]
train_labels = train.target[:train_size]

test_sentences = train.text[train_size:]
test_labels = train.target[train_size:]

In [26]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)
#associer à chaque mot un index

In [27]:
word_index = tokenizer.word_index

In [35]:
word_index

{'co': 1,
 'http': 2,
 'like': 3,
 'fire': 4,
 'amp': 5,
 'get': 6,
 'not': 7,
 'via': 8,
 'new': 9,
 'burn': 10,
 'go': 11,
 'peopl': 12,
 'u': 13,
 'news': 14,
 'flood': 15,
 'one': 16,
 'emerg': 17,
 'bomb': 18,
 'w': 19,
 'bodi': 20,
 'year': 21,
 'build': 22,
 'crash': 23,
 'time': 24,
 'fatal': 25,
 'video': 26,
 'disast': 27,
 'attack': 28,
 'p': 29,
 'evacu': 30,
 'day': 31,
 'x': 32,
 'home': 33,
 'kill': 34,
 'love': 35,
 'would': 36,
 'say': 37,
 'r': 38,
 'bag': 39,
 'derail': 40,
 'collaps': 41,
 'scream': 42,
 'us': 43,
 'death': 44,
 'polic': 45,
 'look': 46,
 'famili': 47,
 'c': 48,
 'b': 49,
 'see': 50,
 'e': 51,
 'know': 52,
 'make': 53,
 'drown': 54,
 'come': 55,
 'man': 56,
 'want': 57,
 'first': 58,
 'n': 59,
 'train': 60,
 'watch': 61,
 'california': 62,
 'live': 63,
 'world': 64,
 'hijack': 65,
 'accid': 66,
 'back': 67,
 'still': 68,
 'got': 69,
 'two': 70,
 'work': 71,
 'car': 72,
 'obliter': 73,
 'take': 74,
 'destroy': 75,
 'let': 76,
 'full': 77,
 'fear': 78

In [28]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [29]:
from tensorflow.keras.utils import pad_sequences

train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)
#Car pour que LSTM marche, il faut qu'on ait des séquences de même longueur

In [37]:
train_sequences[0]

[3512, 507, 310, 92, 1289, 2749, 43]

In [39]:
train_padded[0]

array([3512,  507,  310,   92, 1289, 2749,   43,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [30]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [44]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()

model.add(Embedding(num_words, 32, input_length=max_length))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))


model.compile(loss='binary_crossentropy', optimizer='adam')

In [45]:
from tensorflow.keras.callbacks import EarlyStopping

In [46]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',patience=3)

In [47]:
history = model.fit(
    train_padded, train_labels, epochs=20, validation_data=(test_padded, test_labels),callbacks=[early_stop]
)

Epoch 1/20
191/191 [==============================] - 7s 18ms/step - loss: 0.5286 - val_loss: 0.4366
Epoch 2/20
191/191 [==============================] - 3s 15ms/step - loss: 0.3006 - val_loss: 0.5245
Epoch 3/20
191/191 [==============================] - 3s 16ms/step - loss: 0.1641 - val_loss: 0.6289
Epoch 4/20
191/191 [==============================] - 3s 17ms/step - loss: 0.0940 - val_loss: 0.9298
